In [22]:
#Library imports
import re
import os
import shutil
import time
import math
import fnmatch
import random
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
#import matplotlib.font_manager as fm

from IPython.display import Image, display
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from PIL import Image, ImageDraw

from tensorflow.keras import models, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras import initializers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, LSTM, Dropout, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.regularizers import l1, l2


#User Created functions

from cleaning_functions import *
from eda_functions import *
from modeling_functions import *
from setup_functions import *

from random_lumberjacks.src.random_lumberjacks.model.model_classes import *
from random_lumberjacks.src.random_lumberjacks.visualization.visualization_functions import *

#Notebook arguments
%matplotlib inline

In [2]:
#Without this block the Japanese font's won't display properly in Matplotlib.Set to your font directory.
extend_matplotlib_fonts("/usr/share/fonts/")

/home/anacuda/work/nyc-mhtn-ds-021720/japanese_text_classifiers/setup_functions.py:23: MatplotlibDeprecationWarning: 
The createFontList function was deprecated in Matplotlib 3.2 and will be removed two minor releases later. Use FontManager.addfont instead.
  font_list = fm.createFontList(font_files)


In [3]:
#Device specific gpu deterministic arguments
from tensorflow import config as tfconfig
physical_devices = tfconfig.list_physical_devices('GPU')
tfconfig.experimental.set_memory_growth(physical_devices[0], enable=True)
os.environ['TF_DETERMINISTIC_OPS'] = '1'

In [4]:
#Sets random seeds to allow for reproducable results.
from tensorflow import random as tfrandom
SEED=127
os.environ['PYTHONHASHSEED']=str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tfrandom.set_seed(SEED)

Since there are many classes, I've imported a json that contains useful information about the characters and stored it into a DataFrame.

In [13]:
hiragana_classes = pd.read_json("choubenkyo_kivy_app/app/draw_screen/characters/hiragana.json")

# By converting the recorded stroke count to a list of integers, it will be easier to iterate through the data and remove obvious mistakes from the start.
hiragana_classes["stroke_count"] = hiragana_classes["stroke_count"].map(lambda x: [int(item[0]) for item in re.finditer("\d+", x)])

# It will be useful to have seperate dataframes to account for dropped classes whether they are the obsolete kana or the compound characters.
current_hiragana_classes = hiragana_classes[(hiragana_classes["char_id"]!= "wi") & (hiragana_classes["char_id"]!= "we")].reset_index(drop=True)
reduced_hiragana_classes = current_hiragana_classes.iloc[0:46]
compound_hiragana_classes = current_hiragana_classes.iloc[46:]

## Creating Bitmaps

The first goal is to convert all the raw coordinate data collected from the phone app into 28 x 28 greyscale images to match the other datasets. The Pillow library has a function that can render these points to a bitmap image. I've created several functions that ensure that the images are rendered to be centered/filled to the image space.

In [20]:
def render_coordinates_file_to_img(path, resolution=(28, 28), stroke_width=2):
    coord_dict = load_pickle(path)
    adjusted_coords = parse_to_points_list(coord_dict)
    scaled = scale_points_for_pixels(adjusted_coords, resolution, stroke_width)
    img = Image.new('L', resolution, color=0)
    draw = ImageDraw.Draw(img)
    for coords in scaled:
        line_from_array(draw, coords, width=stroke_width)
    return img

def create_classification_dirs(path_list, labels):
    for path in path_list:
        for label in labels:
            os.makedirs(os.path.join(path, label))

Then in order to loop through and QC/arrange the data in an efficient manner a few paths need to be stored as variables.

In [11]:
raw_path = "data/choubenkyo_data/raw"
rendered_path = "data/choubenkyo_data/rasterized"
rendered_split_paths = {"train": os.path.join(rendered_path, "train"),
                        "val": os.path.join(rendered_path, "val"),
                        "test": os.path.join(rendered_path, "test")}

In [21]:
create_classification_dirs(rendered_split_paths.values(), reduced_hiragana_classes["char_id"])

In [37]:
# Reproducible results
np.random.seed(SEED)

#create_classification_dirs(rendered_split_paths.values(), reduced_hiragana_classes["char_id"])

pathmap = []
for source in os.listdir(raw_path):
    old_dir = os.path.join(raw_path, source)
    contents = os.listdir(old_dir)
    for label in reduced_hiragana_classes["char_id"]:
        matches = fnmatch.filter(contents, f"{label}*")
        np.random.shuffle(matches)
        match_splits = (image_path_list_train_test_split(matches, .7, .1))
        for i,(key, value) in enumerate(rendered_split_paths.items()):
            new_dir = os.path.join(value, label)
            for match in match_splits[i]:
                raw_path = os.path.join(old_dir, match)
                raw_strokes = load_pickle(raw_path)
                print(match, raw_strokes)
                new_path = os.path.join(new_dir, match)


a-cor_0017 {1: array([[-0.62264878,  0.39329877],
       [-0.61037668,  0.38917536],
       [-0.41696862,  0.3418951 ],
       [-0.29928524,  0.32116296],
       [-0.1575602 ,  0.2967754 ],
       [-0.03813881,  0.28383871],
       [ 0.09631806,  0.28046481],
       [ 0.21897041,  0.28846187],
       [ 0.32810806,  0.31156522],
       [ 0.42626942,  0.35615975],
       [ 0.48568719,  0.40383933]]), 2: array([[-0.17594307,  0.76853668],
       [-0.17594307,  0.76853668],
       [-0.17481146,  0.76190008],
       [-0.17061146,  0.71791185],
       [-0.17055044,  0.63054317],
       [-0.17575123,  0.52956037],
       [-0.18379945,  0.42449962],
       [-0.1990692 ,  0.27467303],
       [-0.22306546,  0.10731248],
       [-0.25812007, -0.22454595],
       [-0.25812007, -0.32876872],
       [-0.25898585, -0.38989354],
       [-0.26836946, -0.44485732],
       [-0.3107976 , -0.51106645]]), 3: array([[ 0.175943  ,  0.13189726],
       [ 0.1179137 ,  0.05535285],
       [-0.01708176, -0.178302

In [51]:
render_coordinates_file_to_img("data/choubenkyo_data/raw/batch01/pya-cor_0007", stroke_width=1)

In [108]:
new_data_list = parse_to_points_list(data)
new_data = scale_points_for_pixels(new_data_list, buffer=2)

In [116]:
im = Image.new('L', (28, 28), color=0)
draw = ImageDraw.Draw(im)
line_from_array(draw, new_data[0])
line_from_array(draw, new_data[1])
line_from_array(draw, new_data[2])
line_from_array(draw, new_data[3])
im

In [37]:
draw